In [ ]:
from box_auto.utils import WS
import rosbag
from tf_bag import BagTfTransformer
from tf.transformations import quaternion_from_euler, quaternion_multiply
from math import pi
from pathlib import Path
import yaml
import copy

# Compare extrinsics
path = Path(WS) / "src/grand_tour_box/box_calibration/box_calibration/calibration/"

frame_ids = [
    "alphasense_front_center",
    "alphasense_front_left",
    "alphasense_front_right",
    "alphasense_left",
    "alphasense_right",
    "hdr_front",
    "hdr_left",
    "hdr_right",
    "zed2i_right",
    "zed2i_left"
]

res = {}

for p in sorted([str(s) for s in path.glob("*tf_static_metadata_new_*.bag")]):
    print(p)
    tf_listener = BagTfTransformer(p)
    res[p] = {}
    for frame_id in frame_ids:
        tf = tf_listener.lookupTransform("box_base", frame_id, time=None, latest=True)
        res[p][frame_id] = tf
        
    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Number of subplots based on frame_ids
n_subplots = len(frame_ids)

# Create subplots with shared y-axis and same scale for x-axis
fig, ax = plt.subplots(n_subplots, 1, figsize=(10, 2*n_subplots), sharex=True)

# Iterate through each subplot
for i in range(n_subplots):
    # Create lineplot
    x_axis = [n for n in range(0, len(res))]  # X-axis based on the range of res

    ref_position = np.array( list(res.values())[0][frame_ids[i]][0])
    print(ref_position, list(res.values())[0][frame_ids[i]][0])
    y_axis = [np.linalg.norm(ref_position - np.array(list(res.values())[j][frame_ids[i]][0])) for j in range(len(res))]  # Y-axis data from res
    
    ax[i].plot(x_axis, y_axis)
    ax[i].set_title(f"Position offset over time in [m]:  {frame_ids[i]}")
    ax[i].set_xlabel("Metadata number")
    ax[i].set_ylabel("L2 Norm Position")
    ax[i].set_ylim(min(y_axis)*0.999,max(y_axis)*1.0001)
    
# Improve layout and display
plt.tight_layout()
plt.show()